In [120]:
from pprint import pp
import re
from string import ascii_uppercase

sample_data_mini = """start-a
start-b
b-end
a-end
"""

sample_data = """start-A
start-b
A-c
A-b
b-d
A-end
b-end"""

sample_data2 = """dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc"""

sample_data3 = """
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW"""



def get_data(sample):
    pairs = re.findall(r'(\w+)-(\w+)', sample)
    data = {}
    for x,y in pairs:
        data.setdefault(x, []).append(y)
        data.setdefault(y, []).append(x)
    return data



pp(get_data(sample_data))
pp(get_data(sample_data2))
pp(get_data(sample_data3))



{'start': ['A', 'b'],
 'A': ['start', 'c', 'b', 'end'],
 'b': ['start', 'A', 'd', 'end'],
 'c': ['A'],
 'd': ['b'],
 'end': ['A', 'b']}
{'dc': ['end', 'start', 'HN', 'LN', 'kj'],
 'end': ['dc', 'HN'],
 'HN': ['start', 'dc', 'end', 'kj'],
 'start': ['HN', 'kj', 'dc'],
 'kj': ['start', 'sa', 'HN', 'dc'],
 'LN': ['dc'],
 'sa': ['kj']}
{'fs': ['end', 'he', 'DX', 'pj'],
 'end': ['fs', 'zg'],
 'he': ['DX', 'fs', 'pj', 'RW', 'WI', 'zg'],
 'DX': ['he', 'start', 'pj', 'fs'],
 'start': ['DX', 'pj', 'RW'],
 'pj': ['DX', 'zg', 'he', 'RW', 'start', 'fs'],
 'zg': ['end', 'sl', 'pj', 'RW', 'he'],
 'sl': ['zg'],
 'RW': ['he', 'pj', 'zg', 'start'],
 'WI': ['he']}


In [121]:
DEBUG = True
def get_crap(data):
    results = []
    def recurse(node: str, visited: list, depth: int):
        nonlocal results
        pad = " " * depth
        depth+=1
        DEBUG and print(pad + f"Received {node}.  Been to {visited}")
        eligible = [d for d in data[node] if d not in visited or d.isupper()]
        DEBUG and print(pad + f"Can go to {eligible}")
        for path in eligible:
            branched = visited.copy()
            branched.append(node)
            if path != 'end':
                DEBUG and print(pad + f"Recursing: {branched} to {path}")
                recurse(path, branched, depth)
            else:
                branched.append(path)
                DEBUG and print(pad + f"End: {branched}")
                results.append(branched)

    recurse('start', [], 0)
    return results



def do(sample_data):
    data = get_data(sample_data)
    results = sorted(get_crap(data))
    print(len(results))

do(sample_data)
DEBUG = False

Received start.  Been to []
Can go to ['A', 'b']
Recursing: ['start'] to A
 Received A.  Been to ['start']
 Can go to ['c', 'b', 'end']
 Recursing: ['start', 'A'] to c
  Received c.  Been to ['start', 'A']
  Can go to ['A']
  Recursing: ['start', 'A', 'c'] to A
   Received A.  Been to ['start', 'A', 'c']
   Can go to ['b', 'end']
   Recursing: ['start', 'A', 'c', 'A'] to b
    Received b.  Been to ['start', 'A', 'c', 'A']
    Can go to ['A', 'd', 'end']
    Recursing: ['start', 'A', 'c', 'A', 'b'] to A
     Received A.  Been to ['start', 'A', 'c', 'A', 'b']
     Can go to ['end']
     End: ['start', 'A', 'c', 'A', 'b', 'A', 'end']
    Recursing: ['start', 'A', 'c', 'A', 'b'] to d
     Received d.  Been to ['start', 'A', 'c', 'A', 'b']
     Can go to []
    End: ['start', 'A', 'c', 'A', 'b', 'end']
   End: ['start', 'A', 'c', 'A', 'end']
 Recursing: ['start', 'A'] to b
  Received b.  Been to ['start', 'A']
  Can go to ['A', 'd', 'end']
  Recursing: ['start', 'A', 'b'] to A
   Received A

In [124]:
do(sample_data2)

19


In [125]:
do(sample_data3)

226


In [126]:
real_data = open('d12.input').read()
data = get_data(real_data)
results = sorted(get_crap(data))
print(len(results))



4792


In [157]:
DEBUG = True
small_cave = re.compile(r'^[a-z]{1,2}$').match
from collections import Counter

def get_different_crap(data):
    results = []
    def recurse(node: str, visited: list, depth: int):
        nonlocal results
        pad = " " * depth
        depth+=1
        eligible = []
        DEBUG and print(pad + f"Received {node}.  Been to {visited}")
        for neighbor in data[node]:
            if neighbor not in visited:
                eligible.append(neighbor)
            elif neighbor.isupper():
                eligible.append(neighbor)
            elif len([v for v in visited + [node] if v == neighbor and small_cave(v)]) == 1:
                if 2 not in Counter(v for v in visited + [node] if small_cave(v)).values():
                    eligible.append(neighbor)
        DEBUG and print(pad + f"Can go to {eligible}")
        for path in eligible:
            branched = visited.copy()
            branched.append(node)
            if path != 'end':
                DEBUG and print(pad + f"Recursing: {branched} to {path}")
                recurse(path, branched, depth)
            else:
                branched.append(path)
                DEBUG and print(pad + f"End: {branched}")
                results.append(branched)

    recurse('start', [], 0)
    return results



def do_second(sample_data):
    data = get_data(sample_data)
    results = sorted(get_different_crap(data))
    print(len(results))
    return results
DEBUG = False
results = do_second(sample_data)
# pp(results)
# for r in results:
#     print(r)
#     print(Counter(v for v in r if small_cave(v)))
DEBUG = False

36


In [153]:
results = do_second(sample_data2)

      Received HN.  Been to ['start', 'HN', 'dc', 'kj', 'sa', 'kj']
      Can go to ['end']
      End: ['start', 'HN', 'dc', 'kj', 'sa', 'kj', 'HN', 'end']
103


In [154]:
results = do_second(sample_data3)

3509


In [158]:
results = do_second(open('d12.input').read())

133360
